# Goal

Goal of the notebook is to train and resnet150 model and store oof preds on validation set in /oof_preds folder.

In [1]:
%matplotlib inline

In [2]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from sklearn import metrics

In [5]:
PATH='../data/raw/HE_DL/'
arch=resnet152
sz=350; bs=32

In [6]:
label_csv=f'{PATH}meta-data/train.csv'
n=len(list(open(label_csv))) - 1
val_idxs=get_cv_idxs(n)
n, len(val_idxs)

(13000, 2600)

## Initial Model

In [7]:
def get_data(sz, bs, val_idxs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.05)
    data = ImageClassifierData.from_csv(PATH, 
                                        'train/', 
                                        f'{PATH}meta-data/train.csv',
                                        bs=bs,
                                        tfms=tfms,
                                        val_idxs=val_idxs, # to make sure only single image is there in validation set
                                        test_name='test/'
                                       )
    
    return data if sz > 300 else data.resize(500, 'tmp/')

## Full Training

In [20]:
val_idxs = [0]

data  = get_data(sz, bs, val_idxs)
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 1/1 [00:00<00:00, 22.51it/s]


In [21]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.206808   0.001037   1.0       
    1      0.158312   0.000293   1.0                          



[array([ 0.00029]), 1.0]

In [22]:
learn.precompute=False
learn.fit(1e-2, 5, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.154492   3e-05      1.0       
    1      0.122927   1.4e-05    1.0                         
    2      0.127206   2.2e-05    1.0                         
    3      0.107906   4e-06      1.0                         
    4      0.08981    1.2e-05    1.0                          



[array([ 0.00001]), 1.0]

In [ ]:
learn.set_data(get_data(450, bs=32,val_idxs=val_idxs))
learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.110427   2e-06      1.0       
    1      0.111339   2e-06      1.0                          
    2      0.089514   2e-06      1.0                          



[array([ 0.]), 1.0]

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.100742   0.0        1.0       
    1      0.091426   2e-06      1.0                          
    2      0.082444   0.0        1.0                          
    3      0.08803    0.0        1.0                          
 81%|████████▏ | 331/407 [06:45<01:33,  1.23s/it, loss=0.071] 

## Create Submission

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs        = np.mean(np.exp(log_preds),0)

In [ ]:
df         = pd.DataFrame(probs)
df.columns = data.classes

df.insert(0, 'image_id', [o.split('/')[1] for o in data.test_ds.fnames])
df.loc[:, 'img_num'] = [int(f.split('-')[1].split('.')[0]) for f in data.test_ds.fnames]

df = df.sort_values(by='img_num')
df.drop('img_num', axis=1, inplace=True)

df.to_csv('../data/raw/HE_DL/submissions/sub16.csv', index=False)